<a href="https://colab.research.google.com/github/kahramankaya/VeriBilimiKucukProje/blob/main/Colaboratory'ye_Ho%C5%9F_Geldiniz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [151]:
import pandas as pd
import torch

verileri okuyup biraz gözden geçirelim.

In [152]:
df=pd.read_csv("/kaggle/input/social-network-ads/Social_Network_Ads.csv")
df.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [153]:
df.columns

Index(['User ID', 'Gender', 'Age', 'EstimatedSalary', 'Purchased'], dtype='object')

In [154]:
df.shape

(400, 5)

In [155]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
User ID,400.0,1.569154e+07,71658.321581,15566689.0,15626763.75,15694341.5,15750363.0,15815236.0
Age,400.0,3.765500e+01,10.482877,18.0,29.75,37.0,46.0,60.0
EstimatedSalary,400.0,6.974250e+04,34096.960282,15000.0,43000.00,70000.0,88000.0,150000.0
Purchased,400.0,3.575000e-01,0.479864,0.0,0.00,0.0,1.0,1.0


normalizasyon yapmadan önce verilerde Null deger yani boş değer olup olmadığına bakalım. varsa onları ortalama ile doldurabiliriz.

In [156]:
df.isnull().sum()

User ID            0
Gender             0
Age                0
EstimatedSalary    0
Purchased          0
dtype: int64

In [157]:
y=df.loc[:,"Purchased"]
y.head(10)

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    1
8    0
9    0
Name: Purchased, dtype: int64

**burda cinsiyet columnu numeric yaptım.**

In [158]:
df=pd.get_dummies(df,"Gender",dtype=int)
df

,User ID,Age,EstimatedSalary,Purchased,Gender_Female,Gender_Male
0,15624510,19,19000,0,0,1
1,15810944,35,20000,0,0,1
2,15668575,26,43000,0,1,0
3,15603246,27,57000,0,1,0
4,15804002,19,76000,0,0,1
...,...,...,...,...,...,...
395,15691863,46,41000,1,1,0
396,15706071,51,23000,1,0,1
397,15654296,50,20000,1,1,0
398,15755018,36,33000,0,0,1


**Age ve EstimatedSalary column larına normalizasyon uyguladım**

In [159]:
x_min=df["Age"].min()
x_max=df["Age"].max()
Age_norm=(df["Age"]-x_min)/(x_max-x_min)
df["Age"]=Age_norm

In [160]:
y_min=df["EstimatedSalary"].min()
y_max=df["EstimatedSalary"].max()
Salary_norm=(df["EstimatedSalary"]-y_min)/(y_max-y_min)
df["EstimatedSalary"]=Salary_norm

**ve normalize olmus datamız hazır şekilde .**

In [161]:
df.drop("User ID",axis="columns",inplace=True)
df

,Age,EstimatedSalary,Purchased,Gender_Female,Gender_Male
0,0.023810,0.029630,0,0,1
1,0.404762,0.037037,0,0,1
2,0.190476,0.207407,0,1,0
3,0.214286,0.311111,0,1,0
4,0.023810,0.451852,0,0,1
...,...,...,...,...,...
395,0.666667,0.192593,1,1,0
396,0.785714,0.059259,1,0,1
397,0.761905,0.037037,1,1,0
398,0.428571,0.133333,0,0,1


In [162]:
indep_col=['Age','EstimatedSalary', 'Gender_Female',"Gender_Male"]
dep_col=["Purchased"]

bagımsız columnları ve bagımlıyı ayrı ayrı degıskenlere atayalım kı gradıent descent uygulayabılelım

In [163]:
df_indep=df[indep_col]
df_dep=df[dep_col]

**bağımsız değişkenlerimiz**

In [164]:
df_indep

,Age,EstimatedSalary,Gender_Female,Gender_Male
0,0.023810,0.029630,0,1
1,0.404762,0.037037,0,1
2,0.190476,0.207407,1,0
3,0.214286,0.311111,1,0
4,0.023810,0.451852,0,1
...,...,...,...,...
395,0.666667,0.192593,1,0
396,0.785714,0.059259,0,1
397,0.761905,0.037037,1,0
398,0.428571,0.133333,0,1


**bağımlı değişkenlerimiz**

In [165]:
df_dep

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


 ** bagımlı ve bagımsız degıskenlerımızı tensore atayalım kı daha sonra işlem yapalım. **

In [166]:
from torch import tensor 
tensor_indep=tensor(df_indep.values,dtype=torch.float)
tensor_dep=tensor(df_dep.values)
tensor_indep

tensor([[0.0238, 0.0296, 0.0000, 1.0000],
        [0.4048, 0.0370, 0.0000, 1.0000],
        [0.1905, 0.2074, 1.0000, 0.0000],
        ...,
        [0.7619, 0.0370, 1.0000, 0.0000],
        [0.4286, 0.1333, 0.0000, 1.0000],
        [0.7381, 0.1556, 1.0000, 0.0000]])

In [167]:
tensor_indep.shape

torch.Size([400, 4])

** 4 column oldugu ıcın 4 tane 0 ve 1 arasında katsayı uretelım ve daha sonra 0.5 ten cıkartalım . böylece katsayılar -0.5 ve 0.5 arasında olacak **

In [168]:
random_num=torch.rand(4,requires_grad=True)
coeff=random_num-0.5
random_num,coeff

(tensor([0.2368, 0.1853, 0.9876, 0.8136], requires_grad=True),
 tensor([-0.2632, -0.3147,  0.4876,  0.3136], grad_fn=<SubBackward0>))

** bunlar bızım tahmınlerımız. daha sonra bu tahminleri loss hesaplama da kullanacağız. **

In [169]:
def calc_preds(coeffs,df_indep):
    preds=(coeff*tensor_indep).sum(axis=1)
    return preds
    
calc_preds(coeff,df_indep)

tensor([ 2.9798e-01,  1.9536e-01,  3.7219e-01,  3.3329e-01,  1.6510e-01,
         1.5692e-01,  2.7034e-01,  8.5145e-02,  2.2773e-01,  2.6450e-01,
         2.8594e-01,  3.5121e-01,  1.3552e-01,  2.1883e-01,  1.5738e-01,
         9.3093e-02,  1.0849e-01,  1.1869e-01,  1.0776e-01,  2.6694e-01,
         1.2802e-01,  2.2658e-01,  6.4923e-02,  3.0206e-01,  1.1942e-01,
         1.2015e-01,  8.8961e-02,  2.7087e-01,  1.7935e-01,  2.2509e-01,
         9.4545e-02,  1.4679e-01,  4.6647e-01,  3.5733e-01,  8.2317e-02,
         1.7904e-01,  3.6329e-01,  1.5909e-01,  3.0459e-01,  3.9390e-01,
         4.2654e-01,  3.0967e-01, -9.7880e-03,  2.3836e-01,  2.6408e-01,
         2.7057e-01,  1.2050e-01,  3.4028e-01, -4.1392e-02,  2.3362e-01,
         4.1037e-01,  4.2000e-01,  2.6014e-01,  3.6241e-01,  3.3096e-01,
         3.5675e-01,  3.7934e-01,  1.0169e-01,  2.8150e-01,  1.6208e-01,
         2.4550e-01,  1.0185e-01,  3.3738e-01, -1.8959e-02,  7.2106e-02,
         1.7572e-01,  2.6664e-01,  3.0008e-01,  3.5

ilk başta random 4 katsayı belirlemiştik ve bunları kullanırsak loss bu imiş.

In [170]:
def calc_loss(coeff,tensor_indep):
    preds=calc_preds(coeff,tensor_indep)
    loss=torch.abs((preds-tensor_dep)).mean()
    return loss
calc_loss(coeff,tensor_indep)

tensor(0.4114, grad_fn=<MeanBackward0>)

başlangıc ıcın katsayılarımızı oluşturacak bir fonksiyon yaptım

In [171]:
def init_coeff():
    random_num=torch.rand(4,requires_grad=True)
    coeff=random_num-0.5
    return coeff.requires_grad_()

**bize her epoch ta katsayılarımızı güncelleyecek bir fonksiyon lazım.**

In [172]:
def update_coeff(coeff,lr):
    coeff.sub_(coeff.grad*lr)
    coeff.grad.zero_()

In [173]:
def one_epoch(coeff,lr=0.01):
    loss = calc_loss(coeff, tensor_indep)
    loss.backward()
    with torch.no_grad(): update_coeff(coeff, lr)
    print(f"loss: {loss:.3f}", end="; ")
    

In [174]:
type(coeff)

torch.Tensor

In [175]:
def train_model(epochs):
    torch.manual_seed(442)
    coeff=init_coeff()
    for i in range(epochs): one_epoch(coeff, lr=0.01)
    return coeff
    
train_model(15)

/tmp/ipykernel_47/75729554.py:2: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /usr/local/src/pytorch/build/aten/src/ATen/core/TensorBody.h:486.)
  coeff.sub_(coeff.grad*lr)


TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'

In [ ]:
# import necessary libraries 
import torch 

# define a tensor 
A = torch.tensor(5., requires_grad=True) 
print("Tensor-A:", A) 

# define a function using above defined 
# tensor 
x = A**3
print("x:", x) 

# print the gradient using .grad 
print("A.grad:", A.grad) 


In [ ]:
# import necessary libraries 
import torch 

# define a tensor 
A = torch.tensor(5., requires_grad=True) 
print("Tensor-A:", A) 

# define a function using above defined 
# tensor 
x = A**3
print("x:", x) 

# call the backward method 
x.backward() 

# print the gradient using .grad 
print("A.grad:", A.grad) 


In [ ]:
def fy(x):
    print(x)
    
def fx(x=5):
    fy(x)
    
fx(x=5)